In [23]:
import pandas as pd
import requests
import json
from pandasql import sqldf

## ACS American Community Survey, conducted once a year provides population counts for varying demographics, age, race, veteran status, gender etc
## Maybe there is a better way to get this data,code loops through years and demographic groups to dowload data
## Each letter A,B,C etc refers to a different demographic 
## remove 2020 because of covid 19, no reporting

## Script will download definitions first, clean data and then download the values 

get_years = list(range(2007, 2021+1))
## No Data from 2020 due to Covid 19
get_years.remove(2020)

get_letters = ['A','B','C','D','E','F','G','H','I']

# Define the API endpoint URL
# Loop through A,B,C, etc

link_acs_values = 'https://api.census.gov/data/xxxx/acs/acs1?get=NAME,group(B21001zzzz)&for=state:*'


In [ ]:
link_acs_definitions = 'https://api.census.gov/data/2021/acs/acs1/groups/B21001zzzz/'

##Initialize a blank DF to add definitions to

acs_definitions = pd.DataFrame()

for letter in get_letters:
    tmp_link = link_acs_definitions.replace('zzzz',letter)
    ## Request definitions, currently assuming definitions are same for all years
    tmp_definitions = requests.get(tmp_link)
    tmp_definitions = tmp_definitions.json()
    tmp_definitions = pd.DataFrame(tmp_definitions['variables']).transpose()
    tmp_definitions['variable'] = tmp_definitions.index
    acs_definitions = pd.concat([acs_definitions,tmp_definitions], axis = 0)
acs_definitions

In [ ]:

## Split acs_definitions label by delimitter !!
def_tmp = acs_definitions['label'].str.split('!!', expand = True)
def_tmp.columns = ['col1','col2','col3','col4','col5']
def_tmp['variable'] = def_tmp.index
def_tmp


In [ ]:
## Extract Characteristics into unique columns, the gender for example is not always in the same column because each variable does not always have the same number of characteristics
#

def_tmp = sqldf("""
SELECT variable,
       col1 AS metric,
       col2 AS total,
       CASE
           WHEN lower(col3) LIKE '%male%' THEN col3
           WHEN lower(col4) LIKE '%male%' THEN col4
           WHEN lower(col5) LIKE '%male%' THEN col5
       END AS gender,
       CASE
           WHEN lower(col3) LIKE '%veteran%' THEN col3
           WHEN lower(col4) LIKE '%veteran%' THEN col4
           WHEN lower(col5) LIKE '%veteran%' THEN col5
       END AS veteran_status,
       CASE
           WHEN lower(col3) LIKE '%to%' OR lower(col3) LIKE '%over%' THEN col3
           WHEN lower(col4) LIKE '%to%' OR lower(col4) LIKE '%over%' THEN col4
           WHEN lower(col5) LIKE '%to%' OR lower(col5) LIKE '%over%' THEN col5
       END AS age_group
FROM def_tmp
""")
def_tmp.set_index('variable', inplace=True)

##Clean up identifiers and replace blanks with all
def_tmp['gender'] = def_tmp['gender'].str.rstrip(':')
def_tmp['veteran_status'] = def_tmp['veteran_status'].str.rstrip(':')
def_tmp['age_group'] = def_tmp['age_group'].str.rstrip(':')

def_tmp['gender'].fillna('All', inplace=True)
def_tmp['veteran_status'].fillna('All', inplace=True)
def_tmp['age_group'].fillna('All', inplace=True)

def_tmp

In [ ]:
## Extract race from concept column - after "(" and before ")"
acs_definitions['race'] = acs_definitions['concept'].str.extract(r"\((.*)\)")
acs_definitions


In [ ]:
## Add back cleaned columns from def_tmp
acs_definitions = pd.concat([acs_definitions,def_tmp], axis = 1)
acs_definitions

In [24]:
## Download acs data will take a long time if getting for all years
acs_values = pd.DataFrame()
for year in get_years:
    for letter in get_letters:
        ##print(year)
        ##print(letter)
        link_tmp = link_acs_values.replace('xxxx',str(year))
        link_tmp = link_tmp.replace('zzzz',letter)
        values_tmp = requests.get(link_tmp)
        values_tmp = values_tmp.json()
        values_tmp = pd.DataFrame(values_tmp)
        values_tmp.columns = values_tmp.iloc[0]
        ## For some reason NAME comes down twice from census need to skip first column to avoid, skip first row to make header first row
        values_tmp = values_tmp.iloc[1: , 1:]
        values_tmp = pd.melt(values_tmp, id_vars=['NAME','GEO_ID','state'], var_name="variable", value_name="value")
        values_tmp['year'] = year
        acs_values = pd.concat([acs_values, values_tmp], axis = 0, ignore_index=True)
    
acs_values.columns = ['state','geo_id', 'stateID', 'variable', 'value', 'year']
acs_values

2007
2007
2007
2007
2007
2007
2007
2007
2007
2008
2008
2008
2008
2008
2008
2008
2008
2008
2009
2009
2009
2009
2009
2009
2009
2009
2009
2010
2010
2010
2010
2010
2010
2010
2010
2010
2011
2011
2011
2011
2011
2011
2011
2011
2011
2012
2012
2012
2012
2012
2012
2012
2012
2012
2013
2013
2013
2013
2013
2013
2013
2013
2013
2014
2014
2014
2014
2014
2014
2014
2014
2014
2015
2015
2015
2015
2015
2015
2015
2015
2015
2016
2016
2016
2016
2016
2016
2016
2016
2016
2017
2017
2017
2017
2017
2017
2017
2017
2017
2018
2018
2018
2018
2018
2018
2018
2018
2018
2019
2019
2019
2019
2019
2019
2019
2019
2019
2021
2021
2021
2021
2021
2021
2021
2021
2021


,NAME,GEO_ID,state,variable,value,year
0,Alabama,0400000US01,01,B21001A_001E,2521614,2007
1,Alaska,0400000US02,02,B21001A_001E,344418,2007
2,Arizona,0400000US04,04,B21001A_001E,3681379,2007
3,Arkansas,0400000US05,05,B21001A_001E,1723197,2007
4,California,0400000US06,06,B21001A_001E,16796748,2007
...,...,...,...,...,...,...
864859,Washington,0400000US53,53,B21001I_033MA,None,2021
864860,West Virginia,0400000US54,54,B21001I_033MA,None,2021
864861,Wisconsin,0400000US55,55,B21001I_033MA,None,2021
864862,Wyoming,0400000US56,56,B21001I_033MA,None,2021


In [31]:
acs_definitions.to_csv('acs_definitions.csv', index=False)

NameError: name 'acs_definitions' is not defined

In [30]:
acs_values.to_csv('acs_values_state.csv', index=False)